# The lapidarist problem (data cleansing)
As many of the real world data the data available is not always correct and ready to use, therefore before even moving to EDA and model implementation clean the data to be ready to analyze and then ready to use is key. In this section of the work I cleaned the data and I offer a brief explantion on each step of the cleansing process. 

In [2]:
# File extraction
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns


diamonds_df = pd.read_csv('data/diamonds.csv')
coords_diamonds_df = pd.read_csv('data/coords_diamonds.csv')

A first look at each table allows to see that I needed to merge both tables using the "Unnamed: 0" feature as the joining argument. I also tested that the number of observations matched between tables. 

In [3]:
print(diamonds_df.shape)
diamonds_df.head()

(53930, 11)


,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
print(coords_diamonds_df.shape)
coords_diamonds_df.head()

(53930, 3)


,Unnamed: 0,latitude,longitude
0,0,39.8813889,-83.093056
1,1,33.415,-111.548889
2,2,40.1672222,-105.101389
3,3,37.3711111,-85.583056
4,4,48.5394444,-121.745000


In [5]:
# Merging both data sets
diamonds_df = diamonds_df.merge(coords_diamonds_df, how = 'left', on = 'Unnamed: 0')
diamonds_df.drop('Unnamed: 0', inplace = True, axis = 1)

By merging table the predictive model will use information from 11 different features of +50K diamonds to predict as accurately as possible the price of Krenk's missing values. 

I'm not an diamond expert. However, some of the variables are well self explantory for the others I made a quick research on what they were in the context of diamonds, the variables that I did not found on the research are taken as another input to support the model. 

The 11 different features we are going to use to try to predict are: 
* carat: the weight of the diamond
* cut: the quality of the diamond's cut, which affects its brilliance
* color: the color grade of the diamond, indicating how colorless it is
* clarity: the clarity grade of the diamond, which determines de present of internal 
* depth: the depth percentage of the diamond
* table: the table percentage of the diamond
* price: the price of the variable (this is the dependent variable that will be called y)
* x, y and z: likely represent the lenght, width and depth of the diamond in milimeters
* latitude and lingitud : some spacial information (no more info)

### Data Quality Report
On the DQR I am able to quickly identify the possible data cleaning steps to set the data correctly.

In [6]:
# Function lib
def dqr(data):
    """
    Generate a Data Quality Report
    """
    cols = pd.DataFrame(list(data.columns.values),
                           columns=['Names'],
                           index=list(data.columns.values))
    dtyp = pd.DataFrame(data.dtypes,columns=['Type'])
    misval = pd.DataFrame(data.isnull().sum(),
                                  columns=['Missing_values'])
    presval = pd.DataFrame(data.count(),
                                  columns=['Present_values'])
    unival = pd.DataFrame(columns=['Unique_values'])
    minval = pd.DataFrame(columns=['Min_value'])
    maxval = pd.DataFrame(columns=['Max_value'])
    for col in list(data.columns.values):
        unival.loc[col] = [data[col].nunique()]
        try:
            minval.loc[col] = [data[col].min()]
            maxval.loc[col] = [data[col].max()]
        except:
            pass
    
    return cols.join(dtyp).join(misval).join(presval).join(unival).join(minval).join(maxval)

def is_number(s):
    """
    Identify if a value is number or not
    """
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def clean_string(string):
    """
    Eliminate all non alpha numeric characters
    """
    try:
        string=  [char for char in string if char.isalnum() and char]
        return ''.join(string)
    except ValueError:
        return False
    
def save_cleaned_data(data, file_path):
    """
    Save cleaned data to a CSV file.
    """
    data.to_csv(file_path, index=False)

From the dqr we can make the following first conclusions. 

1. There's missing data in 3 fields (depth, x and y). Although the sample relative to the present values does not exceed the 5% a Data Scientist normally have 2 options to handle missing values, either I input the missing data with the mean or median of the actual computed values or  delete those observations from the data set. 

2. There are inputing errors in the categorical values (cut, color, clarity). The data cleanning can be very exhaustive but things can be simplified if you find the correct pattern to correct those inputs. 

3. x, y and z are variables likely to represent the size of the diamond... it is weird to see that we have negative values on such features. 

In [7]:
print(tabulate(dqr(diamonds_df), headers='keys', tablefmt='pretty', showindex=False))

+-----------+---------+----------------+----------------+---------------+-------------+------------+
|   Names   |  Type   | Missing_values | Present_values | Unique_values |  Min_value  | Max_value  |
+-----------+---------+----------------+----------------+---------------+-------------+------------+
|   carat   | float64 |       0        |     53930      |      273      |     0.2     |    5.01    |
|    cut    | object  |       0        |     53930      |      26       |    !Good    | Very Good  |
|   color   | object  |       0        |     53930      |      26       |     !E      |     J      |
|  clarity  | object  |       0        |     53930      |      30       |    !VS2     |    VVS2    |
|   depth   | float64 |      2962      |     50968      |      184      |    43.0     |    79.0    |
|   table   | float64 |       0        |     53930      |      127      |    43.0     |    95.0    |
|   price   |  int64  |       0        |     53930      |     11602     |     326     |   1

### Data cleansing
I started the pre processing by addressing the first issues we saw on the dqr. The first thing I cleaned is the values wrongly coded in latitude. You can see that there's only one diamond that is incorrectly inputed because it contains a q besides the decimal separator; so I replaced the incorrect value "q" by "" and the converted that number to float as the rest of coordinates. 

In [8]:
# Identify which diamonds contained errors in their latitude
diamonds_df[diamonds_df['latitude'].apply(is_number) == False]

,carat,cut,color,clarity,depth,table,price,x,y,z,latitude,longitude
48185,0.7,Fair,F,SI2,64.6,57.0,1947,5.6,5.57,3.61,33q.200088,-105.624152


In [9]:
# Changing the identified observations
diamonds_df['latitude'] = diamonds_df['latitude'].str.replace("q", "").astype('float64')

After I corrected the latitude I moved to the rest of categorical values. As I mentioned early, by finding the correct cleanning pattern you can save a lot of time. In order to achieve that, I first looked for all the different possible classes of each categorical variables, after inspection I realized that the only pattern I needed to apply was that each character in the class must be an alphanumerical value. 

After finding the pattern I created a function to strip all the non alphanumerical characters and I got my correct set of classes which will be helpfull to correctly encode this variables later in the solution. 

In [10]:
# Printing all unique values for each categorical variable
for i in ['cut', 'color', 'clarity']:
    print("Unique values for ", i, "var:")
    print(diamonds_df[i].unique())



Unique values for  cut var:
['Ideal' 'Premium' 'Good' 'Very Good' 'Fair' 'Very Goo!d' "P*'remium"
 'I#deal' '#Very Good' 'P?remium' "*'Ideal" '!Good' 'Pre!mium' 'Pr?emium'
 'Very Go#od' 'Ide&al' 'Ide!al' 'Id!eal' '&Premium' 'Go?od' 'G#ood'
 "Very *'Good" "Ide*'al" 'V&ery Good' '&Ideal' 'Very G#ood']
Unique values for  color var:
['E' 'I' 'J' 'H' 'F' 'G' 'D' "*'E" '#F' '&G' '!H' '?J' '#D' '&I' '!I' '?G'
 '#H' '?H' '?E' "*'F" '!E' '&F' '#G' "*'I" '!F' '&E']
Unique values for  clarity var:
['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF' 'S?I1' 'SI!1' '&VS2'
 '&SI2' "S*'I1" 'VS?1' "S*'I2" '#VS1' 'V&S2' 'V!S2' '!VS2' 'VS#2' "VVS*'2"
 "*'SI2" 'VV?S1' 'S&I1' "*'SI1" 'SI?1' 'VV#S1' 'V#S2' '#SI!1' 'S!I2']


In [11]:
# Application of clean string function
var_st = diamonds_df[['cut', 'color', 'clarity']].applymap(lambda string: clean_string(string))

# Checking again the unique values to accept correction
for i in ['cut', 'color', 'clarity']:
    print("Unique values for ", i, "var:")
    print(var_st[i].unique())

# Replacing the correct values into the original dataset
diamonds_df[['cut', 'color', 'clarity']] = var_st

Unique values for  cut var:
['Ideal' 'Premium' 'Good' 'VeryGood' 'Fair']
Unique values for  color var:
['E' 'I' 'J' 'H' 'F' 'G' 'D']
Unique values for  clarity var:
['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF']


As i also mentioned, it seems incorrect that variables (a priori) related to the size of the diamond contain negatives value, this fact along with the very few instances of this phenomenon allows us to elaborate the hypothesis that those values should actually be positive values of that magnitude so I converted those instances to positive using absolute value function. 

In [12]:
# Converting the incorrect numbers into correct abs ones
diamonds_df[['x', 'y', 'z']] = abs(diamonds_df[['x', 'y', 'z']])

Finally I'll save in the data folder the cleaned data so this process is not repeated after each model tryout

In [13]:
save_cleaned_data(diamonds_df, 'data/cleaned_data.csv')